# Different Solutions for 1-Dimensional-1-Phase Problem

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://drive.google.com/file/d/1-5ztISn_pEuNjBWLEckyimFk1pDJf9wc/view?usp=drive_link)

*Author: Zakariya Abugrin | Date: May 2025*

In [ ]:
# Colab only:
try:
    # Install reservoirflow in Colab
    import os
    import google.colab
    !pip install reservoirflow
    # Restart session after installation
    os.kill(os.getpid(), 9)
    print("Session was restarted.")
    print("Now you can run to the following cells.")
except ImportError:
    pass

## Introduction

The solution for a simple 1-dimensional-1-phase (1D1P) problem is available in all solutions which includes: `analytical`, `numerical`, `neurical`. In tutorial, we will see how we can build all solutions and compare them together. 

## Prerequisites:
Before you can follow this tutorial, you need to understand the following tutorials:
1. [Understand scalers Module](/user_guide/tutorials/tutorial_understand_scalers_module.html).
2. [Understand Compiling](/user_guide/tutorials/tutorial_understand_compiling.html).

## Import `reservoirflow`

We start with importing `reservoirflow` as `rf`. The abbreviation `rf` refers to `reservoirflow` where all modules under this library can be accessed. `rf` is also used throughout the [documentation](/api/API.html). We recommend our users to stick with this convention.

In [1]:
import reservoirflow as rf
import numpy as np

print(rf.__version__)

0.1.0


## Create a Model

In [2]:
run_config = dict(
    nsteps=100,
    threading=True,
    vectorize=True,
    check_MB=True,
    print_arrays=False,
    isolver=None,  #'cgs',
)

In [3]:
def create_model(nx=101, n_w=2, comp=False):
    if comp:
        g_comp = 1 * 10**-6
        f_comp = 1 * 10**-5
    else:
        g_comp = None
        f_comp = None

    grid = rf.grids.RegularCartesian(
        nx=nx,
        ny=1,
        nz=1,
        dx=300,
        dy=350,
        dz=40,
        phi=0.27,
        kx=270,
        comp=g_comp,
        dtype="double",
    )

    fluid = rf.fluids.SinglePhase(
        mu=0.5,
        B=1,
        comp=f_comp,
        dtype="double",
    )

    model = rf.models.BlackOil(
        grid, fluid, pi=4000, dtype="double", dt=1, verbose=False
    )
    if n_w > 0:
        well_ids = np.cumsum([nx // (n_w + 1) for i in range(n_w)])
        print("Wells_ids:", well_ids)
        for well_id in well_ids:
            # model.set_well(id=well_id, q=-600, s=1.5, r=3.5)
            model.set_well(id=well_id, pwf=1000, s=1.5, r=3.5)
        model.set_boundaries({0: ("pressure", 4000), nx + 1: ("pressure", 4000)})
    else:
        model.set_boundaries({0: ("pressure", 4000), nx + 1: ("pressure", 1000)})
        # model.set_boundaries({0: ("pressure", 1000), nx+1: ("pressure", 4000)})

    return model

In [4]:
model = create_model(n_w=0, comp=True)

In [5]:
model.compile("analytical", "1D1P")
model.run(**run_config)
model.run(**run_config)

[info] D1P1 was assigned as model.solution.
[info] Simulation run started: 100 timesteps.


[step] 1000: 100%|██████████| 1000/1000 [01:02<00:00, 16.10steps/s]


[info] Simulation run of 100 steps finished in 62.13 seconds.
[info] Simulation run started: 100 timesteps.


[step] 1000: 100%|██████████| 1000/1000 [01:02<00:00, 16.11steps/s]

[info] Simulation run of 100 steps finished in 62.11 seconds.


In [6]:
model.solution.tstep

100

In [7]:
print(model.solution.pressures.shape)
model.solution.pressures

(10403, 103)


array([[4000.        , 4000.        , 4000.        , ..., 3989.27771411,
        3950.1025607 , 1000.        ],
       [4000.        , 4000.        , 4000.        , ..., 3989.27771411,
        3950.1025607 , 1000.        ],
       [4000.        , 4000.        , 4000.        , ..., 3989.27771411,
        3950.1025607 , 1000.        ],
       ...,
       [4000.        , 3987.62120047, 3975.22754443, ..., 1094.710814  ,
        1047.36457992, 1000.        ],
       [4000.        , 3987.62120047, 3975.22754443, ..., 1094.710814  ,
        1047.36457992, 1000.        ],
       [4000.        , 3987.62120047, 3975.22754443, ..., 1094.710814  ,
        1047.36457992, 1000.        ]])

In [8]:
df = model.get_df(columns=['cells_pressure'])
df.iloc[-1].values

array([4000.        , 3987.62120047, 3975.22754443, 3962.80418455,
       3950.33629191, 3937.80906523, 3925.20774013, 3912.51759838,
       3899.72397729, 3886.81227908, 3873.76798032, 3860.5766415 ,
       3847.22391657, 3833.69556269, 3819.97744994, 3806.05557125,
       3791.9160523 , 3777.54516163, 3762.92932079, 3748.05511461,
       3732.90930155, 3717.47882423, 3701.75081996, 3685.71263141,
       3669.35181742, 3652.65616382, 3635.61369437, 3618.21268178,
       3600.44165879, 3582.28942927, 3563.74507944, 3544.79798904,
       3525.43784254, 3505.65464042, 3485.43871032, 3464.78071827,
       3443.67167978, 3422.10297094, 3400.06633939, 3377.5539152 ,
       3354.5582216 , 3331.07218558, 3307.08914832, 3282.6028754 ,
       3257.60756679, 3232.09786664, 3206.06887271, 3179.51614561,
       3152.4357176 , 3124.82410116, 3096.67829707, 3067.99580221,
       3038.77461681, 3009.01325134, 2978.71073286, 2947.86661095,
       2916.480963  , 2884.55439906, 2852.08806601, 2819.08365

In [9]:
model.compile("numerical", "FDM", True)
model.run(**run_config)

[info] FDM was assigned as model.solution.
[info] Simulation run started: 100 timesteps.


[step] 100: 100%|██████████| 100/100 [00:00<00:00, 329.72steps/s]

[info] Simulation run of 100 steps finished in 0.3 seconds.
[info] Material Balance Error: 7.602807272633072e-13.


In [10]:
print(model.solution.pressures.shape)
model.solution.pressures

(101, 103)


array([[4000.        , 4000.        , 4000.        , ..., 4000.        ,
        4000.        , 1000.        ],
       [4000.        , 4000.        , 4000.        , ..., 2044.30826038,
        1415.49650699, 1000.        ],
       [4000.        , 3999.99999999, 3999.99999998, ..., 1619.25974855,
        1211.73326543, 1000.        ],
       ...,
       [4000.        , 3993.79351118, 3981.37292285, ..., 1072.05192067,
        1024.0205196 , 1000.        ],
       [4000.        , 3993.69696822, 3981.08335228, ..., 1071.68835969,
        1023.89928132, 1000.        ],
       [4000.        , 3993.60118203, 3980.7960529 , ..., 1071.33055675,
        1023.77996359, 1000.        ]])

In [11]:
model.compile("numerical", "FDM", False)
model.run(**run_config)

[info] FDM was assigned as model.solution.
[info] Simulation run started: 100 timesteps.


[step] 100: 100%|██████████| 100/100 [00:00<00:00, 167.98steps/s]

[info] Simulation run of 100 steps finished in 0.6 seconds.
[info] Material Balance Error: 7.600586826583822e-13.


In [12]:
print(model.solution.pressures.shape)
model.solution.pressures

(101, 103)


array([[4000.        , 4000.        , 4000.        , ..., 4000.        ,
        4000.        , 1000.        ],
       [4000.        , 4000.        , 4000.        , ..., 2044.30826038,
        1415.49650699, 1000.        ],
       [4000.        , 3999.99999999, 3999.99999998, ..., 1619.25974855,
        1211.73326543, 1000.        ],
       ...,
       [4000.        , 3993.79351118, 3981.37292285, ..., 1072.05192067,
        1024.0205196 , 1000.        ],
       [4000.        , 3993.69696822, 3981.08335228, ..., 1071.68835969,
        1023.89928132, 1000.        ],
       [4000.        , 3993.60118203, 3980.7960529 , ..., 1071.33055675,
        1023.77996359, 1000.        ]])

In [26]:
model.solutions

{'BlackOil-analytical-D1P1-True': <reservoirflow.solutions.analytical.d1p1.D1P1 at 0x22a82c46f90>,
 'BlackOil-numerical-FDM-True': <reservoirflow.solutions.numerical.fdm.FDM at 0x22afa20a250>,
 'BlackOil-numerical-FDM-False': <reservoirflow.solutions.numerical.fdm.FDM at 0x22a8284a790>}

In [14]:
print(model.solutions['BlackOil-numerical-FDM-False'].nsteps)
print(model.solutions['BlackOil-numerical-FDM-True'].nsteps)

101
101


In [15]:
repr(model.compiler)

"rf.solutions.Compiler(model='BlackOil', stype='numerical', method='FDM', sparse=False)"

In [16]:
repr(model.solution)

'<reservoirflow.solutions.numerical.fdm.FDM object at 0x0000022A8284A790>'

In [17]:
model.set_solution('BlackOil-numerical-FDM-True')
repr(model.solution)

'<reservoirflow.solutions.numerical.fdm.FDM object at 0x0000022AFA20A250>'

In [18]:
model.compiler

rf.solutions.Compiler(model='BlackOil', stype='numerical', method='FDM', sparse=True)

In [19]:
# eval(repr(model.compiler))

In [22]:
help(model.run)

Help on method run in module reservoirflow.solutions.analytical.d1p1:

run(nsteps=10, threading=True, vectorize=True, check_MB=True, print_arrays=False, isolver=None) method of reservoirflow.solutions.analytical.d1p1.D1P1 instance
    Solve multiple timesteps.
    
    .. attention::
        This is an abstract method.



In [23]:
model.run(**run_config)

[info] Simulation run started: 100 timesteps.


[step] 1000: 100%|██████████| 1000/1000 [00:59<00:00, 16.90steps/s]

[info] Simulation run of 100 steps finished in 59.21 seconds.


In [24]:
import numpy as np


def logarithmic_scaling(x, min_range, max_range, c=1):
    scaled_value = (
        (np.log(x + c) - np.log(X.min() + c))
        / (np.log(X.max() + c) - np.log(X.min() + c))
    ) * (max_range - min_range) + min_range
    return scaled_value


# Example usage
X = np.array([1, 10, 100, 1000])

min_range = 0  # Replace with your desired minimum range
max_range = 1  # Replace with your desired maximum range

scaled_X = logarithmic_scaling(X, min_range, max_range)

print(scaled_X)


def inverse_logarithmic_scaling(scaled_value, min_range, max_range, c=1):
    original_value = (
        np.exp(
            (
                (scaled_value - min_range)
                * (np.log(X.max() + c) - np.log(X.min() + c))
                / (max_range - min_range)
            )
            + np.log(X.min() + c)
        )
        - c
    )
    return original_value


# Example usage
scaled_X = np.array([0, 0.5, 0.8, 1])  # Replace with your scaled values
min_range = 0  # Replace with your specified minimum range
max_range = 1  # Replace with your specified maximum range

original_X = inverse_logarithmic_scaling(scaled_X, min_range, max_range)

print(original_X)

[0.         0.27426894 0.63098792 1.        ]
[   1.           43.74371464  287.77079009 1000.        ]


```{include} /_static/comments_section.md
```